In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore('./.cache/')

# Token 방식으로 텍스트를 인식
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",                 # 특정 기준으로 분할
    chunk_size=600,
    chunk_overlap=100,
)

# UnstruturedFileLoader : 많은 파일들을 불러올 수 있음. docx, xlsx, pdf, ppt, txt 등 많은 파일들을 불러올 수 있는 Loader
loader= UnstructuredFileLoader("./files/대입합불정리_세특포함_디랩.pdf")

docs = loader.load_and_split(text_splitter=splitter)                                                                                                                                           


embeddings = OpenAIEmbeddings()

cacehd_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 캐시를 적용하여 임베딩
# vectorstore = Chroma.from_documents(docs, cacehd_embeddings)
vectorstore = FAISS.from_documents(docs, cacehd_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 훌륭한 조수입니다. 주어진 자료만을 이용하여 질문에 대답하세요. 만약, 답을 모른다면 모른다고 하세요. 모르는 정보는 지어내지 마세요. \n\n{context}"),
    ("human", "{question}")
])

# RunnablePassthrough() : 입력을 말 그대로 통과시켜줌
chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("성균관대를 합격자의 평균 내신등급은 어느 정도야 ?")

AIMessage(content='성균관대를 합격한 학생들의 평균 내신은 3.40입니다.')